In [2]:
import glob
print(glob.glob("/share/*"))

from torch import nn as nn
import torch
import faiss
from torch_cluster import knn_graph
from timeseries.timeseries_transformer import TimeseriesDataloader, TimeseriesTransformerConfig, DistancePositionalEmbedding, PositionalEmbedding,TimeseriesPandasTrainer
from t3_karpathy.commons.feed_forwards import GeluFeedForward
from t3_karpathy.enhanced_karpathy_transformer import BlockSequence

from t3_karpathy.commons.commons import AbstractCodec, AbstractAccumulativeTrainer, AbstractRunner, \
    BaseTransformerConfig, TimeseriesFeedForward

from torch_cluster import knn
print(torch.cuda.is_available())
directory_path = '/share/us-stock-dataset/Data/Stocks'


['/share/us-stock-dataset', '/share/Transformers1']
False


In [3]:

stocks_to_load = [
    "AAPL", "TSLA",
    "A", "GOOG", "AMZN", "PYPL", "NVDA", "AMD",
    "NFLX", "MSFT", "INTC", "CSCO", "ADBE", "CRM", "QCOM", "TXN", "AVGO",
    "INTU", "ORCL", "COST", "SBUX", "AMGN", "CHTR", "GILD", "CMCSA", "BKNG",
    "MDLZ", "FISV", "BIIB", "MU", "MCD", "AMAT", "ADP", "ILMN", "ATVI", "ISRG",
    "ADSK", "LRCX", "BIDU", "JD", "REGN", "WBA", "VRTX", "KHC", "WMT", "ZM", "MELI",
    "TMUS", "CTSH", "XLNX", "PCAR", "ALGN", "WDAY", "SIRI", "CTXS", "ADI", "EXC", "LULU",
    "MAR", "KLAC", "PAYX", "EA", "ILMN", "ALXN", "MNST", "BMRN", "EBAY", "CTAS", "VRSK",
    "IDXX", "CDNS", "NXPI", "ASML", "INCY", "KLAC", "MCHP", "SNPS", "SWKS", "VRSN",
    "WDC", "WYNN", "XLNX", "ZBRA", "ZTS", "AEP", "AIG", "ALL", "AXP", "BA", "BAC",
    "BK", "BLK", "C", "CAT", "CL", "COF", "COP", "COST", "CSCO", "CVS", "CVX",
    "DD", "DHR", "DIS", "DOW", "DUK", "EMR", "EXC", "F", "FDX", "GD", "GE", "GILD",
    "GM", "GOOG", "GOOGL", "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "KHC", "KMI",
    "KO", "LLY", "LMT", "LOW", "MA", "MCD", "MDLZ", "MDT", "MET", "MMM",
    'BLND', 'BLNG', 'BLNKW', 'BLNGU', 'BLNGW', 'BLNK', 'BLPH', 'BLRX',
    'BLTE', 'BLU', 'BLUA', 'BLUE', 'BLW', 'BLX', 'BLZE', 'BMA', 'BMAC', 'BMAQ', 'BMAQR', 'BMAQU'
]


In [4]:
dataloader = TimeseriesDataloader(directory_path, stocks_to_load, add_diff=False)

Reading and merging CSV files: ['AAPL', 'TSLA', 'A', 'GOOG', 'AMZN', 'PYPL', 'NVDA', 'AMD', 'NFLX', 'MSFT', 'INTC', 'CSCO', 'ADBE', 'CRM', 'QCOM', 'TXN', 'AVGO', 'INTU', 'ORCL', 'COST', 'SBUX', 'AMGN', 'CHTR', 'GILD', 'CMCSA', 'BKNG', 'MDLZ', 'FISV', 'BIIB', 'MU', 'MCD', 'AMAT', 'ADP', 'ILMN', 'ATVI', 'ISRG', 'ADSK', 'LRCX', 'BIDU', 'JD', 'REGN', 'WBA', 'VRTX', 'KHC', 'WMT', 'ZM', 'MELI', 'TMUS', 'CTSH', 'XLNX', 'PCAR', 'ALGN', 'WDAY', 'SIRI', 'CTXS', 'ADI', 'EXC', 'LULU', 'MAR', 'KLAC', 'PAYX', 'EA', 'ILMN', 'ALXN', 'MNST', 'BMRN', 'EBAY', 'CTAS', 'VRSK', 'IDXX', 'CDNS', 'NXPI', 'ASML', 'INCY', 'KLAC', 'MCHP', 'SNPS', 'SWKS', 'VRSN', 'WDC', 'WYNN', 'XLNX', 'ZBRA', 'ZTS', 'AEP', 'AIG', 'ALL', 'AXP', 'BA', 'BAC', 'BK', 'BLK', 'C', 'CAT', 'CL', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD'

RuntimeError: No CUDA GPUs are available

In [ ]:
data=dataloader.get_data().transpose(0, 1).cuda()
data.shape

In [ ]:
training_data = torch.diff(data, dim=1)


In [ ]:
training_data.std(dim=1)

In [5]:
def min_max_scaling(tensor):
    return (tensor - tensor.min()) / (tensor.max() - tensor.min())


In [6]:
training_data = min_max_scaling(training_data)
print(training_data.shape)

NameError: name 'training_data' is not defined

In [7]:
window_size = 10

def extract_sliding_windows(tensor, window_size):        
    windows = tensor.unfold(1, window_size, 1)
    return windows

sliding_windows=extract_sliding_windows(training_data, window_size)

print(sliding_windows.shape)


NameError: name 'training_data' is not defined

In [8]:
flat_sliding_windows = sliding_windows.reshape(-1, sliding_windows.size(2))
print(flat_sliding_windows.shape)


NameError: name 'sliding_windows' is not defined

In [9]:
flat_sliding_windows.squeeze(1).std()

NameError: name 'flat_sliding_windows' is not defined

In [10]:

class KarpathyTransformerModel(nn.Module):

    def __init__(self, config: TransformerConfig):
        super().__init__()
        self.config = config
        self.token_embedding_table = nn.Embedding(config.vocab_size, config.n_embed)

        # self.pos_emb1 = PositionalEmbedding(config)
        # self.pos_dist_emb = DistancePositionalEmbedding(config)
        self.pos_ffwd = GeluFeedForward(config.n_embed * 3, config.n_embed, config.n_embed * 2, config.dropout)
        self.pos_ln = nn.LayerNorm(config.n_embed * 2)

        self.blocks = BlockSequence(config)
        self.ln_f = nn.LayerNorm(config.n_embed)  # final layer norm
        self.lm_head = nn.Linear(config.n_embed, config.vocab_size)

    def forward_vs_target(self, idx, targets):
        logits = self.forward(idx)

        b, t, c = logits.shape
        logits_view = logits.view(b * t, c)
        targets = targets.view(b * t)
        loss = F.cross_entropy(logits_view, targets)

        return logits_view, loss

    def forward(self, idx):
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)

        x = tok_emb # + pos_emb  # (B,T,C)
        b, t, c = x.shape

        pos_dist_emb = None #self.pos_dist_emb(b)

        pos_emb = None #self.pos_emb1(b, t)

        # pos_emb = torch.cat([pos_emb_dist, pos_emb], dim=-1)
        #
        # pos_emb = self.pos_ffwd(pos_emb)
        #
        # pos_emb = self.pos_ln(pos_emb)
        # pos_emb = pos_emb_dist + pos_emb

        # x, st_pos_emb = self.blocks(x, pos_emb)  # (B,T,C)
        x, st_pos_emb = self.blocks(x, pos_emb, pos_dist_emb)  # (B,T,C)

        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)
        return logits

dataloader = TimeseriesDataloader(directory_path, stocks_to_load)
config = TimeseriesTransformerConfig(
    batch_size=32,
    block_size=512,
    n_embed=32,
    n_head=4,
    n_layer=8,
    kernel_size=1,
    learning_rate=1e-3,
    channels=dataloader.get_number_of_channels()
)
model = KarpathyTransformerModel(config)
trainer1 = EnhancedKarpathyRunner(config, dataloader=dataloader, model=model)

trainer1.train_eval(20000)


NameError: name 'TransformerConfig' is not defined